In [4]:
import mlflow
import matplotlib.pyplot as plt
from mlflow import MlflowClient
from mlflow.entities import ViewType
import pandas as pd
import torch.nn as nn
import pprint
import json
import ast
import sys
sys.path.insert(0, '/novo/projects/departments/cdd/molecular_ai/dpkn/ML-BP-Domino')

from omegaconf import OmegaConf
import seaborn as sns
from pathlib import Path
from hydra import compose, initialize, initialize_config_dir
from src.helpers.dataset import CSVDataLoader
from src.model.composite_model import CompositeModel
from torch.nn.utils.rnn import pad_sequence
import torch
from sklearn.metrics import matthews_corrcoef
from src.helpers.pytorch.light_attention import LightAttentionModule
import numpy as np
from sklearn.metrics import roc_auc_score
import os

In [ ]:
os.environ["AWS_ACCESS_KEY_ID"] = "KEY"
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_SESSION_TOKEN"] = "KEY"

In [7]:
# ALL OF THIS IS FOR AMYPRO22
amypro = pd.read_csv('/novo/projects/departments/cdd/molecular_ai/sbxw/repos/PALM/datasets/amypro22.csv')
labels_df = pd.read_csv('/novo/projects/departments/cdd/molecular_ai/sbxw/repos/PALM/datasets/amypro22_residues.csv')
ldf = labels_df[['name', 'res_value_bool', 'res_idx']]
ldf

,name,res_value_bool,res_idx
0,Prolactin__AP00001,0,0
1,Prolactin__AP00001,0,1
2,Prolactin__AP00001,0,2
3,Prolactin__AP00001,0,3
4,Prolactin__AP00001,0,4
...,...,...,...
4044,Galectin-7__AP00139,1,131
4045,Galectin-7__AP00139,1,132
4046,Galectin-7__AP00139,1,133
4047,Galectin-7__AP00139,1,134


In [8]:
client = MlflowClient(tracking_uri='https://datalab.corp.novocorp.net/mlflow/')

In [9]:
run_id = '9ce7f66eb6f24268b4c941b5db0d9ce0'
os.makedirs(f'../../models_from_mlflow/{run_id}',exist_ok=True)
client.download_artifacts(run_id, dst_path=f'../../models_from_mlflow/{run_id}', path='')

INFO:botocore.credentials:Found credentials in environment variables.


'/novo/projects/departments/cdd/molecular_ai/dpkn/ML-BP-Domino/models_from_mlflow/9ce7f66eb6f24268b4c941b5db0d9ce0/'

In [10]:
config_path = '/novo/projects/departments/cdd/molecular_ai/sbxw/repos/models_from_mlflow/9ce7f66eb6f24268b4c941b5db0d9ce0/'

with initialize_config_dir(config_dir=config_path, version_base=None, job_name="test"):
    cfg = compose(config_name="esm2t68MUR50D_LightAttention_sbxwfibrillationpeptidewaltzdb-5foldtrain-valserrano-testrandpadded10xpadlen-10",
                  overrides=[f"+general.composite_model_path={config_path}",
                             "persistence.data_root_folder=/novo/projects/departments/cdd/molecular_ai/sbxw/repos/PALM/sbxw/datasets/",
                             "persistence.training_data=."])
    print(OmegaConf.to_yaml(cfg))
    model = CompositeModel(cfg)

INFO:src.model.dimred:Load class (NoReduction): NoReduction
INFO:src.model.abstract_components:Load class (DimRed Model): NoReduction
INFO:src.model.predictors:Load class (LightAttention): LightAttention
INFO:src.model.abstract_components:Load class (TorchPredictorModel): LightAttention
INFO:src.model.abstract_components:Load class (Predictor Model): LightAttention
INFO:src.model.predictors:Loading model from /novo/projects/departments/cdd/molecular_ai/sbxw/repos/models_from_mlflow/9ce7f66eb6f24268b4c941b5db0d9ce0/model.pt


general:
  run_mode: test
  abs_repo_path: null
  composite_model_name: esm2t68MUR50D_LightAttention_sbxwfibrillationpeptidewaltzdb-5foldtrain-valserrano-testrandpadded10xpadlen-10
  composite_model_path: /novo/projects/departments/cdd/molecular_ai/sbxw/repos/models_from_mlflow/9ce7f66eb6f24268b4c941b5db0d9ce0/
  precomputed_embeddings_path: null
  random_state: 42
dataset:
  data_name: sbxw_fibrillation_peptide_waltzdb-5fold_train-val_serrano-test_randpadded10x_padlen-10
  add_data_columns: []
  data_columns_dimred: []
  data_columns_standard: []
  group_column: null
  data_scaler: RobustScaler
  target_scaling: null
  rbf_encoder: RadialBasisFunctionGaussian
  rbf_n_kernels: 10
  task: classification_binary
  use_predefined_split: true
  use_sample_weights: false
  residue_prediction_labels: null
  data_split_column: data_split_fold1
  data_split:
    train: 0.8
    val: 0.0
    test: 0.2
embedder:
  class_name: ESM
  model_name: esm2_t6_8M_UR50D
  model_path: null
  standardize: fal

INFO:src.model.composite_model:Composite model name: esm2t68MUR50D_LightAttention_sbxwfibrillationpeptidewaltzdb-5foldtrain-valserrano-testrandpadded10xpadlen-10
INFO:src.model.composite_model:Initialized model: esm2t68MUR50D_LightAttention_sbxwfibrillationpeptidewaltzdb-5foldtrain-valserrano-testrandpadded10xpadlen-10 in run_mode = test
INFO:src.helpers.dataset:Loading file: /novo/projects/departments/cdd/molecular_ai/sbxw/repos/PALM/sbxw/datasets/sbxw_fibrillation_peptide_waltzdb-5fold_train-val_serrano-test_randpadded10x_padlen-10.csv
INFO:src.helpers.dataset:Using group_column "None".
INFO:src.helpers.dataset:Select data according to predetermined splits in the dataframe, train (N = 11190), validation (N = 2800), test (N = 158)
INFO:src.helpers.dataset:Loaded columns: {}


In [11]:
amypro

,Unnamed: 0,sequence,res_value_bool,name,len,dataset,data_split,value_bool
0,13990,LPICPGGAARCQVTLRDLFDRAVVLSHYIHNLSSEMFSEFDKRYTH...,0000001111111111111111111111111111000000001111...,Prolactin__AP00001,199,amypro22,test,1
1,13991,KNTMEHVSSSEESIISQETYKQEKNMAINPSKENLCSTFCKEVVRN...,0000000000000000000000000000000000000000000000...,Alpha-s2-casein__AP00004,207,amypro22,test,1
2,13992,RSFFSFLGEAFDGARDMWRAYSDMREANYIGSDKYFHARGNYDAAK...,1111111111111111111111111110000000000000000000...,Serum_Amyloid_A-1__AP00005,104,amypro22,test,1
3,13994,TQQPQQDEMPSPTFLTQVKESLSSYWESAKTAAQNLYEKTYLPAVD...,0000000000000000000000000000000000000000000000...,Apoliprotein__AP00020,79,amypro22,test,1
4,13995,APLIPQRLMSASNSNELLLNLNNGQLLPLQLQGPLNSWIPPFSGIL...,0000000000000000000000000000000000000000000000...,Odontogenic_ameloblast-associated_protein__AP...,264,amypro22,test,1
5,13996,LIVTQTMKGLDIQKVAGTWYSLAMAASDISLLDAQSAPLRVYVEEL...,0000000000111111111100000000000000000000000000...,Beta-lactoglobulin__AP00025,162,amypro22,test,1
6,13997,GLSDGEWQQVLNVWGKVEADIAGHGQEVLIRLFTGHPETLEKFDKF...,1111111111111111111111111111100000000000000000...,Apomyoglobin__AP00030,153,amypro22,test,1
7,13999,GETDPNTQLLNDLGNNMAWGAALGAPGGLGSAALGAAGGALQTVGQ...,0000000000000000000000000000000000000000000000...,Microcin_E492__AP00040,84,amypro22,test,1
8,14002,MNSLNTQLGANSSFFQVDPGQNTQSSPNQGNQGISEKQLDQLLTQL...,0000000000000000000000000000001111111111111111...,Elicitor_of_hypersensitive_response_HpaG__AP0...,133,amypro22,test,1
9,14003,MALLDFFLSRKKNTANIAKERLQIIVAERRRSDAEPHYLPQLRKDI...,0000000000000000001111111111000000000000000000...,Cell_division_topological_specificity_factor_...,88,amypro22,test,1


In [12]:
amypro['data_split_fold1'] = 'test'
dataloader = CSVDataLoader(cfg, amypro)
dataloader

INFO:src.helpers.dataset:Using group_column "None".
INFO:src.helpers.dataset:Select data according to predetermined splits in the dataframe, train (N = 0), validation (N = 0), test (N = 22)
INFO:src.helpers.dataset:Loaded columns: {}


In [13]:
predictions = model.forward(dataloader)

INFO:src.model.abstract_components:Load class (LLMEmbedderModel): ESM
INFO:src.model.abstract_components:Load class (Embedder Model): ESM
INFO:src.model.embedders:Loading model: esm2_t6_8M_UR50D, will extract embeddings from 6-th layer
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:src.model.embedders:Moving model to cuda
INFO:src.model.abstract_components:Total memory: 23.670685696GB
Memory already allocated: 0.315147776GB
INFO:src.model.embedders:Per-residue embeddings count: 22
INFO:src.model.composite_model:Residue embeddings
INFO:src.model.predictors:Moving embeddings to device: cuda
INFO:src.model.scalers:Before standardization - min: True, max: True
INFO:src.model.scalers:After standardization - min: True, max: True


In [14]:
def apply_mask(output, mask):
    """
    Move predictions to cpu and convert to numpy masked array

    Args:
        output: [batch_size, sequence_length] predictions from the model
        mask: [batch_size, sequence_length] residue mask (true=padding)

    Returns:
        output: [batch_size, sequence_length] array with logits
    """
    output_numpy = output.cpu().numpy()
    mask_numpy = ~mask.cpu().numpy().astype(bool)
    # mask is false by default, otherwise it denotes padding residues
    output_numpy = np.ma.masked_array(output_numpy,mask)
    return output_numpy

In [15]:
mask = pad_sequence([torch.zeros(len(seq)) for seq in amypro.sequence], batch_first=True,padding_value=1.0)
mask

tensor([[0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.]])

In [16]:
masked_output_np = apply_mask(model.predictor.model.o_unflattened,mask)
masked_output_np

masked_array(
  data=[[0.7332018613815308, 0.7384454011917114, 0.7427346706390381, ...,
         --, --, --],
        [0.720990777015686, 0.7312994599342346, 0.7252100706100464, ...,
         --, --, --],
        [0.6899535655975342, 0.6984330415725708, 0.7052538990974426, ...,
         --, --, --],
        ...,
        [0.6901276707649231, 0.7362176775932312, 0.807883620262146, ...,
         --, --, --],
        [0.7134267687797546, 0.7056435942649841, 0.7330008745193481, ...,
         --, --, --],
        [0.7072376012802124, 0.7215359807014465, 0.7307615876197815, ...,
         --, --, --]],
  mask=[[False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]],
  fill_value=1e+20,
  dtype=float32)

In [17]:
masked_output_np_flat = masked_output_np.compressed()
masked_output_np_flat.shape

(4049,)

In [18]:
roc_auc_score(np.array(ldf['res_value_bool']), masked_output_np_flat)

0.6541853740666633